trnsl.1.1.20190208T160544Z.4984aee4447d8d41.9ded34b703cb87362a2a60d529d521163be1c8ae

https://translate.yandex.net/api/v1.5/tr/translate
 ? key=<API key>
 & text=<text to translate>
 & lang=<translation direction>
 & [format=<text format>]
 & [options=<translation options>]

In [7]:
from pathlib import Path
import os
import re
from tqdm import tqdm
from pathlib import Path
import requests
import time

#os.chdir('../')

from src.data import read_sentences

In [48]:
def get_request(api_key, sentence, lang_src, lang_tgt):
    r = requests.get("https://translate.yandex.net/api/v1.5/tr/translate", 
                        data={'key': api_key,\
                            'text': sentence,\
                            'lang': f'{lang_src}-{lang_tgt}',\
                            'options': '4'})
    
    return r

In [58]:
r = get_request(API_KEYS[2], 'hi world lalala', 'en', 'pt')

In [59]:
                tr = re.findall('<text>(.*?)</text>', r.text)
                al = re.findall('<align>(.*?)</align>',r.text)

In [60]:
tr

['oi mundial lalala']

In [61]:
al

['0:15-0:17,0:8-0:10,9:6-11:6']

In [42]:
def translate(path, sentences, api_keys, lang_src='en', lang_tgt='pt'):
    ix_key = 0
    no_tr = []
    translations = []
    alignments = []
    id_translations = []
    with tqdm(total=len(sentences)) as pbar:
        for ix_sentence, sentence in enumerate(sentences):
            try:
                r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
                while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                    print('Changing key')
                    ix_key += 1
                    r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
            except:
                print('sleeping')
                time.sleep(50)
                r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
                while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                    print('Changing key')
                    ix_key += 1
                    r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
                    
            if ix_key >= len(api_keys):
                break
            else:
                tr = re.findall('<text>(.*?)</text>', r.text)
                al = re.findall('<align>(.*?)</align>',r.text)
                if len(tr) and len(al):
                    translations.append(tr[0])
                    alignments.append(al[0])
                    id_translations.append(ix_sentence)
                else:
                    no_tr.append(ix_sentence)
            pbar.update(1)
    
    print('No translated:', no_tr)
    back_translations = []
    for sentence in translations:
        try:
            r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
            while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                ix_key += 1
                r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
        except:
            print('sleeping')
            time.sleep(50)
            r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
            while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                print('Changing key')
                ix_key += 1
                r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
        if ix_key >= len(api_keys):
            break
        else:
            tr = re.findall('<text>(.*?)</text>', r.text)
            if len(tr):
                back_translations.append(tr[0])
                
    or_file = open(path / 'or.txt', 'w')
    trn_file = open(path / 'trn.txt', 'w')
    bck_file = open(path / 'bck.txt', 'w')
    al_file = open(path / 'al.txt', 'w')
    id_file = open(path / 'id.txt', 'w')

    for ix in range(len(id_translations)):
        or_file.write(sentences[ix] + '\n')
        trn_file.write(translations[ix] + '\n')
        bck_file.write(back_translations[ix] + '\n')
        al_file.write(alignments[ix] + '\n')
        id_file.write(id_translations[ix] + '\n')

    or_file.close()
    trn_file.close()
    bck_file.close()
    al_file.close()
    id_file.close()
    
    return id_translations, translations, alignments, back_translations, no_tr

In [3]:
API_KEYS = ['trnsl.1.1.20190214T025847Z.fa65319f70094455.5e6c51a7180828d7d9adcc484b1aec45d0452fe1',
           'trnsl.1.1.20190214T043436Z.15ee3f5f639d23db.232d53490edf34112c2aec6acf83161c31d2cdfb',
           'trnsl.1.1.20190208T160544Z.4984aee4447d8d41.9ded34b703cb87362a2a60d529d521163be1c8ae',
           'trnsl.1.1.20190217T034227Z.98276dd651ef8acb.eacf29edd667aa0789cecf1b88bfb18dbc9eaa8b',
           'trnsl.1.1.20190216T170716Z.034edf2a933741cc.4378585f680c1e47e8bdacd36d2cbdfdd1ec9c2c',
           'trnsl.1.1.20190211T055047Z.c615f5fcf5c0b79e.30465e943c8c4d6222723a66b8a1bc381a9c1f81',
           'trnsl.1.1.20190114T011658Z.9589270a6c96ec20.a04b9a672564f0b41b2a5271aeed8375e4f2c012',
           'trnsl.1.1.20190211T050608Z.273880813ec60ed0.e6bc9d5122fce64de7dacf412e648c1c658e13bc',
           'trnsl.1.1.20190210T185353Z.f5abf55e19682a61.88f40c7fa1dba8fb3eb076700b9bb8c93a0ca44e',
           'trnsl.1.1.20190211T031906Z.bb3fcac7713d5e2e.f0cca830a5c82c1a533729209ac23313c3043e36',
           'trnsl.1.1.20190416T235935Z.39f031ec25aa7b49.9059497d62a297d20c574e7517ce2e5119da4c4e',
           'trnsl.1.1.20190417T000255Z.eb207b405ac31309.7043d5c58aec4959ad1edd00858a9a62cd12022b',
           'trnsl.1.1.20190417T000555Z.24b9a28a589b75f5.deb7d1a519106a3b586ab8d1444f296c4d42b9ca',
           'trnsl.1.1.20190211T210021Z.b390760b52adfb41.8b94e1c488240160fece6452d8c6920b29f24fa6',
           'trnsl.1.1.20190213T225706Z.cf3e02766e4decba.45935fed1c0e9610e5576d802eba55440ef13bbe']

## Reading data

In [5]:
input_sent_semcor, sent_to_id_semcor, target_sent_semcor = read_sentences(Path.cwd() / 'data/WSD_Evaluation_Framework/Training_Corpora/', 'SemCor', False)
len(input_sent_semcor)


37176

In [40]:
input_sent_dev, sent_to_id_dev, target_sent_dev = read_sentences(Path.cwd() / 'data/WSD_Evaluation_Framework/Evaluation_Datasets/', 'ALL', False)
len(input_sent_dev)


1173

## Translation

In [43]:
id_translations, translations, alignments, back_translations, no_tr = translate('data/translation/semcor/en_pt/', 
                                                                         input_sent_semcor, 
                                                                         API_KEYS, 
                                                                         lang_src='en', 
                                                                         lang_tgt='pt')


  0%|          | 0/37176 [00:00<?, ?it/s]

Changing key
Changing key
Changing key
Changing key



  0%|          | 5/37176 [00:10<23:10:16,  2.24s/it]

sleeping


KeyboardInterrupt: 

In [ ]:
id_dev, translations_dev, alignments_dev, back_translations_dev, no_tr_dev = translate('data/translation/dev/en_pt/', 
                                                                         input_sent_dev, 
                                                                         API_KEYS, 
                                                                         lang_src='en', 
                                                                         lang_tgt='pt')


  0%|          | 0/1173 [00:00<?, ?it/s]

Changing key
Changing key
Changing key
Changing key



  1%|          | 11/1173 [00:16<24:43,  1.28s/it]

In [47]:
len(id_dev)

NameError: name 'id_dev' is not defined

In [64]:
path='data/translation/dev/en_pt/'
sentences=input_sent_dev
api_keys=API_KEYS
lang_src='en'
lang_tgt='pt'

In [16]:
len(translations)

37164

In [20]:
translations[-12]

'o kohnstamm-positivo assuntos parecia ser mais livre para experimentar o inusitado e aparentemente impossível no mundo externo .'

In [63]:
len(translations)

1173

In [ ]:
    ix_key = 0
    no_tr = []
    translations = []
    alignments = []
    id_translations = []
    with tqdm(total=len(sentences)) as pbar:
        for ix_sentence, sentence in enumerate(input_sent_dev):
            try:
                r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
                while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                    print('Changing key')
                    ix_key += 1
                    r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
            except:
                print('sleeping')
                time.sleep(120)
                r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
                while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                    print('Changing key')
                    ix_key += 1
                    r = get_request(api_keys[ix_key], sentence, lang_src, lang_tgt)
                    
            if ix_key >= len(API_KEYS):
                break
            else:
                tr = re.findall('<text>(.*?)</text>', r.text)
                al = re.findall('<align>(.*?)</align>',r.text)
                if len(tr) and len(al):
                    translations.append(tr[0])
                    alignments.append(al[0])
                    id_translations.append(ix_sentence)
                else:
                    no_tr.append(ix_sentence)
            pbar.update(1)
    



  1%|          | 14/1173 [00:16<21:53,  1.13s/it]

In [ ]:
    print('No translated:', no_tr)
    back_translations = []
    for sentence in translations:
        try:
            r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
            while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                ix_key += 1
                r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
        except:
            print('sleeping')
            time.sleep(10)
            r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
            while 'Maximum daily translated text volume exceeded' in r.text and ix_key < len(api_keys):
                print('Changing key')
                ix_key += 1
                r = get_request(api_keys[ix_key], sentence, lang_tgt, lang_src)
        if ix_key >= len(API_KEYS):
            break
        else:
            tr = re.findall('<text>(.*?)</text>', r.text)
            if len(tr):
                back_translations.append(tr[0])
                
    or_file = open(path / 'or.txt', 'w')
    trn_file = open(path / 'trn.txt', 'w')
    bck_file = open(path / 'bck.txt', 'w')
    al_file = open(path / 'al.txt', 'w')
    id_file = open(path / 'id.txt', 'w')

    for ix in range(len(id_translations)):
        or_file.write(sentences[ix] + '\n')
        trn_file.write(translations[ix] + '\n')
        bck_file.write(back_translations[ix] + '\n')
        al_file.write(alignments[ix] + '\n')
        id_file.write(id_translations[ix] + '\n')

    or_file.close()
    trn_file.close()
    bck_file.close()
    al_file.close()
    id_file.close()

No translated: []


## Check files

In [154]:
tr_file = open('data/translation/semcor/en_pt/tr.txt', 'r')
or_file = open('data/translation/semcor/en_pt/or.txt', 'r')
bck_file = open('data/translation/semcor/pt_en/bck.txt', 'r')
al_file = open('data/translation/semcor/pt_en/al.txt', 'r')

In [155]:
input_or = []
for sent in or_file:
    input_or.append(sent)

input_tr = []
for sent in tr_file:
    input_tr.append(sent)


In [156]:
bck_input_or = []
for sent in bck_or_file:
    bck_input_or.append(sent)

bck_input_tr = []
for sent in bck_tr_file:
    bck_input_tr.append(sent)    

In [157]:
ix= 0
input_or[ix]

'how long has it been since you reviewed the objectives of your benefit and service program ?\n'

In [158]:
input_tr[ix]

'quanto tempo tem sido desde que reviu os objectivos do seu benefício e serviço de programa ?\n'

In [159]:
bck_input_or[ix]

'quanto tempo tem sido desde que reviu os objectivos do seu benefício e serviço de programa ?\n'

In [162]:
bck_input_tr[ix]

'how long has it been since you reviewed the objectives of your benefit and service program ?\n'

In [161]:
tr_file.close()
bck_tr_file.close()